# Cox ring of the blow up of $\mathbb{P}^3$ in $7$ points. 

This jupyter notebook contains the functions implemented to prove the computational results presented in the article "Cox rings and Mukai edge graphs". In particular, here we present the compuations necessary to prove Theorem 12. 

The code should be read and used together with the paper. 

In [1]:
using Oscar
using Combinatorics

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.10.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


### Auxiliary functions

We beging by introducing some auxillary functions. 

The following two functions compute the intial form of a polynomial and the polynomials in a vector. 
The initial form of a polynomial $f \in \mathbb{C}(t)[z_1,\dots,z_m]$ is defined as 
$$ \text{in}_{{\bf 0}}(f)=(t^{-\text{ord}(f)}f)|_{t=0} \in \mathbb{C}[z_1,\dots,z_m], $$
where $\text{ord}(f)$ is the smallest integer such that $t^{-\text{ord}(f)}f$ has neither a pole or a zero.

In [2]:
function leading_t(eq)
    cont=0
    bol=true
    if length(eq)!=0
        lead=term(eq,1);
        t_lead=degree(term(eq,1),t);
        for mon in range(2,length(eq))
            temp=degree(term(eq,mon),t);
            if temp<t_lead
                bol=true
                 lead=term(eq,mon)
                 t_lead=temp
                 cont=0
            elseif temp==t_lead
                cont=cont+1
                lead=lead+term(eq,mon)
                bol=false
            end
        end
    else
        lead=0
    end
    return lead,bol;
end

function leading_t_vect(eq_vect)
    lead=[]
    bol=true
    for eq in eq_vect
        lead_eq,bol_eq=leading_t(eq)
        push!(lead,lead_eq)
        if !(bol_eq)
            bol=false
        end
    end
    return lead,bol;
end

leading_t_vect (generic function with 1 method)

The following function defines the Veronse map $v_2$ of degree $2$, $v_2: \mathbb{P}^3 \rightarrow \mathbb{P}^9$.

In [35]:
function veronese2(v)
    v_2=KS.([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    cont=1
    for i_1 in 1:4
        for i_2 in 1:4
            if i_1<=i_2
                v_2[cont]=v[i_1]*v[i_2]
                cont=cont+1
            end
        end
    end
    return v_2
end

veronese2 (generic function with 1 method)

The next two functions respectively apply a function $F$ and the deterivative with respect to a variable to a vector of polynomials. 

In [36]:
function sub_vector(Ring,vec,F)
    n = length(vec)
    subs = zeros(Ring,n)
    for i in 1:n 
        subs[i] = F(vec[i])
    end
    return subs
end

function derivative_vector(Ring,vec,var)
    n = length(vec)
    der = zeros(Ring,n)
    for i in 1:n
        der[i] = derivative(vec[i],var)
    end
    return der
end

derivative_vector (generic function with 2 methods)

Given a $4 \times 7$- matrix $A$, we need a function giving linear equations for $\ker A$. Finlly, we need to substitute these equations as first $4$ variables in a polynomial $f \in \mathbb{C}(t)[z_0,\dots,z_6]$. We refer to the section on computing hypersurfaces for more comments on the choice of variables. 

In [37]:
function linear_eqs(A) 
    Lin=zero_matrix(KS,4,1)
    for i in 1:4
        for j in 1:7 
            Lin[i,1]=Lin[i,1]+A[i,j]*(z_vars[j])
        end
    end
    return Lin
end

function sub_linear(A,eqn)
    linear_forms=linear_eqs(A)
    F = hom(KS, KS, z -> z, [t,linear_forms[1,1],linear_forms[2,1], linear_forms[3,1], linear_forms[4,1], z4,z5,z6])
    sub_eqn = F(eqn)
    return sub_eqn
end

sub_linear (generic function with 1 method)

### Generators of $\text{Cox}(X_{7,3})$

Let $A$ be a $4\times 7$ matrix representing $7$ points in $\mathbb{P}(\mathbb{C}(t)^4)$. Given any divisor $D$ of anticanonical degree one and multidegree $(m_0, m_1, \dots, m_7)$, we introduce functions to compute polynomials over $\mathbb{C}(t)$ defining the $129$ hypersurfaces of degree $m_0$ passing through the point $p_i$ with multiplicity $m_i$. 

The polynomials we are computing are in the coordinate ring $\mathbb{C}(t)[z_0, z_1, z_2, z_3]$ of $\mathbb{P}(\mathbb{C}(t)^4)$, but we look at them as polynomials in $\mathbb{C}(t)[z_0, z_1, z_2, z_3, z_4, z_5, z_6]$. We do this for computational reasons and because later on we want to apply the map $\phi_{7,3}(t)$ as defined in Section 3 of the article. 

Each function outputs a polynomial and the multidegree. 

The first function defines the exceptional divisors.

In [38]:
function exceptional()
    exc=zero_matrix(KS,7,1)
    deg_mon_exc=Vector([])
    for i in 1:7
        exc[i,1]=1
        temp=[0, 0, 0, 0, 0, 0, 0, 0]
        temp[i+1]=1
        push!(deg_mon_exc,temp)
    end
    return exc,deg_mon_exc
end

exceptional (generic function with 1 method)

The function hyperplanes computes equations for the $35$ hyperplanes passing through $p_i$, $p_j$ and $p_k$ for distinct indices. They correspond to the divisors $H- E_i -E_j -E_k$. 

In [39]:
function hyperplanes(A)
    Hyp=zero_matrix(KS,binomial(7,3),1)
    deg_mon_hyp=Vector([])
    cont=1
    for i in 1:7
        for j in i+1:7
            for k in j+1:7
                #println("We are doing hyperplane ",i,j,k)
                temp=matrix(KS, [z0 z1 z2 z3 ; transpose(KS.(A)[:,i]); transpose(KS.(A)[:,j]); transpose(KS.(A)[:,k])])
                temp_eq=sub_linear(A,det(temp))
                Hyp[cont,1]=temp_eq
                cont=cont+1
                temp_deg_mon=[1, 1, 1, 1, 1, 1, 1, 1]
                temp_deg_mon[i+1]=0
                temp_deg_mon[j+1]=0
                temp_deg_mon[k+1]=0
                push!(deg_mon_hyp,temp_deg_mon)
            end
        end
    end
    return Hyp,deg_mon_hyp
end

hyperplanes (generic function with 1 method)

The function quadrics computes equations for the $42$ quadric surfaces passing through every point except for $p_i$ and with multipicity two at $p_j$. They correspond to the divisors $2H- \sum_{s\neq i,j} E_s -2E_j$. 

In [40]:
function quadrics(A)
    Quad=zero_matrix(KS,7*6,1)
    deg_mon_quad=Vector([])
    cont_quad=1
    temp_1=transpose(veronese2(KS.([z0,z1,z2,z3])))
    for i in 1:7
        for j in 1:7
            if j!=i
                #println("We are doing quadric ",i,j)
                cont=1
                temp=zero_matrix(KS,10,10)
                temp[1,:]=temp_1[1,:]
                cont=2
                for ind_row in 1:7
                    if (ind_row!=i) & (ind_row!=j) #i is the excluded point
                        F = hom(KS, KS, z -> z, [t,A[1,ind_row], A[2,ind_row], A[3,ind_row], A[4,ind_row],z4,z5,z6])
                        temp[cont,:] = sub_vector(KS,temp_1,F)
                        cont=cont+1
                    end
                end
                G = hom(KS, KS, z -> z, [t,A[1,j], A[2,j], A[3,j], A[4,j],z4,z5,z6]) 
                #j is the double point. So we need to look at the derivates to impose multiplicity two 
                for l in vars 
                    der_l = derivative_vector(KS,temp_1,l)
                    temp[cont,:]=sub_vector(KS,der_l,G)
                    cont=cont+1
                end
                temp_eq=sub_linear(A,det(temp))
                Quad[cont_quad,1]=temp_eq
                cont_quad=cont_quad+1
                temp_deg_mon=[2,1,1,1,1,1,1,1]
                temp_deg_mon[i+1]=2
                temp_deg_mon[j+1]=0
                push!(deg_mon_quad,temp_deg_mon)
            end
        end
    end
    return Quad,deg_mon_quad
end

quadrics (generic function with 1 method)

The function cubics computes equations for the $35$ cubic surfaces passing with multiplicity one at $p_i$, $p_j$ and $p_k$ and with multiplicity two at any other point. They correspond to the divisors $3H- E_i -E_j -E_k - 2\sum_{s\neq i,j, k} E_s.$.

In [41]:
function cubics(A)
    Cubic=zero_matrix(KS,binomial(7,4),1)
    Cubic_z=zero_matrix(KS,binomial(7,4),1)
    deg_mon_cub=Vector([])
    cont_cubic=1
    const_vec=zero_matrix(KS,4,1)
    const_mat=zero_matrix(KS,3,4)
    change_matrix=zero_matrix(KS,4,4)
    points_not_double=zero_matrix(KS,4,3)
    B=zero_matrix(KS,4,3)
    for i in 1:7
        for j in 1:i-1
            for k in 1:j-1
                #println("We are doing cubic ",i,j,k)
                #Construct the matrix for the change of variables
                cont=1
                for p in 1:7
                    if (p!=i) & (p!=j) & (p!=k)
                        change_matrix[:,cont]=A[:,p]
                        cont=cont+1
                    end
                end
                points_not_double[:,3]=A[:,i]
                points_not_double[:,2]=A[:,j]
                points_not_double[:,1]=A[:,k]

                B=adjoint(change_matrix)*points_not_double
         
                #Construction of two constant that allow the cubic to 
                #pass through the two non-double points
            
                const_mat[1,1]=B[1,3]*B[2,3]*B[3,3]
                const_mat[2,1]=B[1,2]*B[2,2]*B[3,2]
                const_mat[3,1]=B[1,1]*B[2,1]*B[3,1]

                const_mat[1,2]=B[1,3]*B[2,3]*B[4,3]
                const_mat[2,2]=B[1,2]*B[2,2]*B[4,2]
                const_mat[3,2]=B[1,1]*B[2,1]*B[4,1]

                const_mat[1,3]=B[1,3]*B[3,3]*B[4,3]
                const_mat[2,3]=B[1,2]*B[3,2]*B[4,2]
                const_mat[3,3]=B[1,1]*B[3,1]*B[4,1]
            
                const_mat[1,4]=B[2,3]*B[3,3]*B[4,3]
                const_mat[2,4]=B[2,2]*B[3,2]*B[4,2]
                const_mat[3,4]=B[2,1]*B[3,1]*B[4,1]
            
                const_vec[1,1]=det(const_mat[:,[2,3,4]])
                const_vec[2,1]=-det(const_mat[:,[1,3,4]])
                const_vec[3,1]=det(const_mat[:,[1,2,4]])
                const_vec[4,1]=-det(const_mat[:,[1,2,3]])

                cubic_temp=const_vec[1,1]*z0*z1*z2+const_vec[2,1]*z0*z1*z3+const_vec[3,1]*z0*z2*z3+const_vec[4,1]*z1*z2*z3

                temp = adjoint(change_matrix)*vars
                F = hom(KS, KS, z -> z, [t,temp[1], temp[2], temp[3], temp[4],z4,z5,z6])
                cubic_temp = F(cubic_temp)
                
                Cubic_z[cont_cubic,1]=cubic_temp
                temp_eq= sub_linear(A,cubic_temp)
                Cubic[cont_cubic,1]=temp_eq
                cont_cubic=cont_cubic+1
                
                temp_deg_mon=[3,1,1,1,1,1,1,1]
                temp_deg_mon[i+1]=2
                temp_deg_mon[j+1]=2
                temp_deg_mon[k+1]=2
                push!(deg_mon_cub,temp_deg_mon)
            end
        end
    end
    return Cubic,deg_mon_cub,Cubic_z
end


cubics (generic function with 1 method)

The function quartics computes equations for the $7$ quartic surfaces passing through all points with multiplicity two and with multiplicity three at $p_i$. They correspond to the divisors $4H- 3E_i - \sum_{s\neq i} 2E_s$.

In order to be able to compute the quartic hypersurfaces for points with coordinates in $\mathbb{C}(t)$ we need to apply a computational trick which is explained in Remark 13 of the article. 

In [42]:
function quartics(A,quad,stra)
    Quart=zero_matrix(KS,7,1)
    deg_mon_quart=Vector([])
    label=[]
    for i in range(1,7)
        for j in range(1,7)
            if i!=j
                push!(label,(i,j))
            end
        end
    end
    ind_0=[2,1,1,1,1,1,1]
    ind_j=[3,3,2,3,4,5,6]
    for i in 1:7
        #println("We are doing quartic ",i)
        Q0i=quad[findfirst(x->x==(ind_0[i],i), label)];
        Q0j=quad[findfirst(x->x==(ind_0[i],ind_j[i]), label)];
        Qji=quad[findfirst(x->x==(ind_j[i],i), label)];
        S0=stra[1];
        S1=stra[2];
        S2=stra[3]
        eq_mono=[S0*Q0i,S1*Q0i,S2*Q0i,Q0j*Qji];
        mono=Set();
        list_mon=[monomials(S0*Q0i),monomials(S1*Q0i),monomials(S2*Q0i),monomials(Q0j*Qji)];
        for l in list_mon
            for el in l
                push!(mono,el);
            end
        end
        M_tar=zero_matrix(FractionField(KS),4,6)
        b=[]
        for cont in range(1,7)
            if cont!=ind_0[i] 
                if cont!=i
                el=[0,0,0,0,0,0,0]
                el[ind_0[i]]=3
                el[cont]=1
                push!(b,el)
                end
            end
        end
        cont=0
        for (el,col) in zip(b,range(1,6))
            for (eq,row) in zip(eq_mono,range(1,4))
                M_tar[row,col]=coeff(eq,[2,3,4,5,6,7,8],el)
            end
        end
        M_tar=transpose(M_tar);
        ker_M_tar=kernel(M_tar)[2];
        #We need to verify the rank
        Quart[i,1]=numerator(sum([eq_mono[i]*ker_M_tar[i,1] for i in range(1,4)]));
        
        temp_deg_mon=[4,2,2,2,2,2,2,2]
        temp_deg_mon[i+1]=1
        push!(deg_mon_quart,temp_deg_mon)
    end
    return Quart,deg_mon_quart;
end

quartics (generic function with 1 method)

The function strange computes equations for sections of $-K_{7,3}/2$ which are quadrcis passing through the seven points. They correspond to the divisors $2H-  \sum_{i} E_i$.

In [43]:
function strange(A,A_plus)
    Strange=zero_matrix(KS,3,1)
    #Strange=zero_matrix(R,3,1)
    deg_mon_strange=Vector([])
    mat_lin_ind=zero_matrix(KS,10,3)
    temp_1=transpose(veronese2(KS.([z0,z1,z2,z3])))
    cont=1
    temp=zero_matrix(KS,10,10)
    for i in 1:3
        temp[1,:]=temp_1[1,:]
        for ind_row in 1:7
            temp[ind_row+1,:]=veronese2(KS.(A[:,ind_row]))
        end      
        for ind_row in 1:2
            temp[ind_row+7+1,:]=A_plus[cont,:]
            cont=cont+1
        end
        #Matrix to verify they are linearly independent
        temp_eq=sub_linear(A,det(temp))
        Strange[i,1]=temp_eq
        for ind in 1:10
            mat_lin_ind[ind,i]=coeff(det(temp),temp_1[1,ind])
        end
        push!(deg_mon_strange,[2,1,1,1,1,1,1,1])
    end
    
    bol=(rank(mat_lin_ind)==3)
    return Strange,deg_mon_strange,bol;
end

strange (generic function with 1 method)

We include here also few check functions. The first one checks that a polynomial is not zero. We use this to confirm that the chosen points are in general position. 

The second one checks that the three hypersurfaces $S_0, S_1, S_2$ computed by the function strange have different initial forms. 

In [44]:
function zero_check(eq)
    for el in eq
        if el==0
            return false
        end
    end
    return true
end

function strange_diff(stra_mon)
    bol=true
    bol1=numerator(monomial(stra_mon[1],1)//t^(degree(stra_mon[1],t)))
    bol2=numerator(monomial(stra_mon[2],1)//t^(degree(stra_mon[2],t)))
    bol3=numerator(monomial(stra_mon[3],1)//t^(degree(stra_mon[3],t)))
    if (bol1==bol2) 
        println("Two monomials are the same")
        bol=false
    end
    if (bol1==bol3) 
        println("Two monomials are the same")
        bol=false
    end
    if (bol2==bol3) 
        println("Two monomials are the same")
        bol=false
    end
    return bol;   
end


strange_diff (generic function with 1 method)

### Khovanksii basis

The function computes the quadratic relations and their multidegrees among the $129$ hypersurfaces that can be computed using the functions in the previous section.

In [45]:
function degree_quadratic(deg_mon,equations)
    monomials_2=[]
    couples=[]
    deg_couples=[]
    for (eq1,i) in zip(equations,range(1,129))
        for (eq2,j) in zip(equations,range(1,129))
            if j>=i
                push!(couples,[eq1,eq2,i,j])
                deg_temp=[deg_mon[i][k]+deg_mon[j][k] for k in range(1,8)]
                push!(deg_couples,deg_temp)
            end
        end
    end
    return couples,deg_couples
end

degree_quadratic (generic function with 1 method)

We now consider the choice of points in the matrix $A$ below. We compute the $129$ polynomials $f_i^D$ in $\mathbb{C}(t)[z_0, z_1, \dots, z_6]$ obtained by applying the functions `exceptional`, `hyperplanes`, `quadrics`, `cubics`, `quartics` and `strange` to $A$. 

For each polynomial $f_iD$ we then look at $f_i^D\left(A\cdot z\right)$ and finally we do the change of variables $z_i \mapsto y_i/x_i$ and the homogeneization to obtain the function $\phi_{7,3}(t)$ described in Section 3. We do this in two steps to make our computations more efficient. 


In [46]:
KS, (t,z0,z1,z2,z3,z4,z5,z6)=PolynomialRing(QQ, ["t","z0","z1","z2","z3","z4","z5","z6"]);
    z_vars = KS.([z0,z1,z2,z3,z4,z5,z6]);
    vars=KS.([z0,z1,z2,z3])

A=[t^59 t^44 t^79 t^20 t^12 t^81 t^36; t^8 t^72 t^49 t^39 t^58 t^23 t^64; t^44 t^58 t^12 t^52 t^57 t^49 t^51; t^25 t^23 t^60 t^72 t^45 t^51 t^6]


4×7 Matrix{fmpq_mpoly}:
 t^59  t^44  t^79  t^20  t^12  t^81  t^36
 t^8   t^72  t^49  t^39  t^58  t^23  t^64
 t^44  t^58  t^12  t^52  t^57  t^49  t^51
 t^25  t^23  t^60  t^72  t^45  t^51  t^6

The matrices below are other examples of points in general position for which we obtain a Khovanskii basis. They were found via a random search. 

In [47]:
#A=[t^136 t^47 t^28 t^101 t^64 t^193 t^83; t^67 t^158 t^46 t^128 t^122 t^133 t^82; t^78 t^183 t^106 t^95 t^78 t^154 t^9; t^98 t^166 t^148 t^35 t^26 t^193 t^95]
#A=[t^13 t^68 t^70 t^27 t^68 t^90 t^10; t^45 t^32 t^9 t^29 t^15 t^13 t^61; t^45 t^82 t^26 t^22 t^9 t^54 t^45; t^2 t^36 t^32 t^25 t^16 t^87 t^68]
#A=[t^96 t^77 t^27 t^21 t^20 t^63 t^76; t^35 t^98 t^80 t^81 t^60 t^85 t^53; t^23 t^22 t^76 t^53 t^99 t^31 t^97; t^27 t^84 t^30 t^79 t^56 t^21 t^74]
#A=[t^58 t^100 t^61 t^11 t^18 t^40 t^100; t^23 t^38 t^40 t^89 t^63 t^56 t^66; t^92 t^25 t^76 t^27 t^67 t^36 t^73; t^18 t^90 t^92 t^73 t^54 t^8 t^18]
#A=[t^178 t^93 t^185 t^97 t^103 t^22 t^32; t^38 t^84 t^177 t^188 t^150 t^192 t^26; t^127 t^14 t^192 t^134 t^23 t^91 t^132; t^173 t^78 t^18 t^155 t^189 t^161 t^19]

In the function we implement the following checks: 
1. The chosen points are in general poistion, by looking at whether any of the $129$ polynomials is zero. 
2. The initial forms of the $129$ polynomials are monomials. 
3. The initial forms of the three polynomials computed by the function strange (sections of $-K_{7,3}/2)$ have different initial forms. 
4. For every multidegree $D$ from quadratic relations the dimension of the corresponding linear system is equal to the number of monomials. 

In [48]:
function is_khovanskii(A)
    KS, (t,z0,z1,z2,z3,z4,z5,z6)=PolynomialRing(QQ, ["t","z0","z1","z2","z3","z4","z5","z6"]);
    z_vars = KS.([z0,z1,z2,z3,z4,z5,z6]);
    vars=KS.([z0,z1,z2,z3])

    deg_2=[[1, 1, 0, 0, 1, 1, 1, 1], [2, 1, 2, 1, 1, 1, 1, 1], [2, 2, 2, 0, 1, 1, 1, 1], [3, 3, 2, 2, 1, 1, 1, 1], [3, 2, 2, 2, 2, 1, 1, 1], [4, 2, 2, 2, 2, 2, 2, 2], [4, 3, 1, 2, 2, 2, 2, 2], [3, 2, 0, 1, 2, 2, 2, 2], [4, 2, 1, 1, 3, 3, 2, 2], [5, 1, 2, 2, 3, 3, 3, 3], [5, 2, 2, 2, 2, 3, 3, 3], [5, 4, 2, 3, 2, 2, 2, 2], [6, 3, 2, 3, 3, 3, 3, 3], [6, 4, 2, 2, 3, 3, 3, 3], [7, 3, 4, 4, 3, 3, 3, 3]]


    exc, deg_mon_exc=exceptional();
    exc_mon,bol_exc=leading_t_vect(exc);

    hyps, deg_mon_hyp= hyperplanes(A);

    if !zero_check(hyps)
        println("One of the hyps is zero")
        return false;
    end
    hyps_mon,bol_hyps= leading_t_vect(hyps);

    if !bol_hyps
        println("The hyperplanes are not monomeric")
        return false;
    end
    println("Hyperplanes check ")

    quad,deg_mon_quad = quadrics(A);
    if !zero_check(quad)
        println("One of the quad is zero")
        return false;
    end
    quad_mon,bol_quad= leading_t_vect(quad);
    if !bol_quad
        println("The quadrics are not monomeric")
        return false;
    end
    println("Quad check ")

    cub,deg_mon_cub = cubics(A);
    if !zero_check(cub)
        println("One of the cub is zero")
        return false;
    end
    cub_mon,bol_cub= leading_t_vect(cub);
    if !bol_cub
        println("The cubics are not monomeric")
        return false;
    end
    println("Cub check ")

    #This is the matrix needed to obtain three different sections
    A_plus=matrix(KS,[0 t^2 0 0  0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 t^3; 0 0 0 0 0 t^4 0 0 0 0; 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 1 0 0 0; 0 0 0 0 0 0 0 0 1 0])

    stra,deg_mon_strange,bol_strange=strange(A,A_plus);
    if !zero_check(stra)
        println("One of the strange is zero")
        return false;
    end

    stra_mon,bol_strange=leading_t_vect(stra);
    if  !bol_strange
        println("The strange sections are not monomeric")
        return false;
    end

    if !strange_diff(stra_mon)
        println("Two initial forms of the strange sections are the same. Change A_bol!")
        return false;
    end
    println("Strange check ")

    quart,deg_mon_quart = quartics(A,quad,stra);
    if !zero_check(quart)
        println("One of the quartics is zero")
        return false;
    end
    quart_mon,bol_quart= leading_t_vect(quart);
    if !bol_quart
        println("The quartics are not monomeric")
        return false;
    end

    hyps_mon=[numerator(el//t^degree(el,t)) for el in hyps_mon]
    quad_mon=[numerator(el//t^degree(el,t)) for el in quad_mon]
    cub_mon=[numerator(el//t^degree(el,t)) for el in cub_mon]
    quart_mon=[numerator(el//t^degree(el,t)) for el in quart_mon]
    stra_mon=[numerator(el//t^degree(el,t)) for el in stra_mon]

    deg_mon=reduce(vcat,([],deg_mon_exc,deg_mon_hyp,deg_mon_quad,deg_mon_cub,deg_mon_quart,deg_mon_strange));
    equations_mon=reduce(vcat, (exc_mon,hyps_mon,quad_mon,cub_mon,quart_mon,stra_mon));
    couples,deg_couples=degree_quadratic(deg_mon,equations_mon);
    sagbi_bol=true

    S_t, (t,z0,z1,z2,z3,z4,z5,z6,e0,e1,e2,e3,e4,e5,e6,H012,H013,H014,H015,H016,H023,H024,H025,H026,H034,H035,
        H036,H045,H046,H056,H123,H124,H125,H126,H134,H135,H136,H145,H146,H156,H234,H235,H236,H245,H246,H256,H345,
        H346,H356,H456,Q01,Q02,Q03,Q04,Q05,Q06,Q10,Q12,Q13,Q14,Q15,Q16,Q20,Q21,Q23,Q24,Q25,Q26,Q30,Q31,Q32,Q34,Q35,
        Q36,Q40,Q41,Q42,Q43,Q45,Q46,Q50,Q51,Q52,Q53,Q54,Q56,Q60,Q61,Q62,Q63,Q64,Q65,C210,C310,C320,C321,C410,C420,
        C421,C430,C431,C432,C510,C520,C521,C530,C531,C532,C540,C541,C542,C543,C610,C620,C621,C630,C631,C632,C640,
        C641,C642,C643,C650,C651,C652,C653,C654,Qr0,Qr1,Qr2,Qr3,Qr4,Qr5,Qr6,S0,S1,S2) = 
        PolynomialRing(QQ, ["t","z0","z1","z2","z3","z4","z5","z6","e0","e1","e2","e3","e4","e5","e6","H012","H013","H014","H015","H016","H023","H024","H025","H026","H034","H035","H036","H045","H046","H056","H123","H124","H125","H126","H134","H135","H136","H145","H146","H156","H234","H235","H236","H245","H246","H256","H345","H346","H356","H456","Q01","Q02","Q03","Q04","Q05","Q06","Q10","Q12","Q13","Q14","Q15","Q16","Q20","Q21","Q23","Q24","Q25","Q26","Q30","Q31","Q32","Q34","Q35","Q36","Q40","Q41","Q42","Q43","Q45","Q46","Q50","Q51","Q52","Q53","Q54","Q56","Q60","Q61","Q62","Q63","Q64","Q65","C210","C310","C320","C321","C410","C420","C421","C430","C431","C432","C510","C520","C521","C530","C531","C532","C540","C541","C542","C543","C610","C620","C621","C630","C631","C632","C640","C641","C642","C643","C650","C651","C652","C653","C654","Qr0","Qr1","Qr2","Qr3","Qr4","Qr5","Qr6","S0","S1","S2"])
    vars_S=S_t.([e0,e1,e2,e3,e4,e5,e6,H012,H013,H014,H015,H016,H023,H024,H025,H026,H034,H035,H036,H045,H046,H056,H123,H124,H125,H126,H134,H135,H136,H145,H146,H156,H234,H235,H236,H245,H246,H256,H345,H346,H356,H456,Q01,Q02,Q03,Q04,Q05,Q06,Q10,Q12,Q13,Q14,Q15,Q16,Q20,Q21,Q23,Q24,Q25,Q26,Q30,Q31,Q32,Q34,Q35,Q36,Q40,Q41,Q42,Q43,Q45,Q46,Q50,Q51,Q52,Q53,Q54,Q56,Q60,Q61,Q62,Q63,Q64,Q65,C210,C310,C320,C321,C410,C420,C421,C430,C431,C432,C510,C520,C521,C530,C531,C532,C540,C541,C542,C543,C610,C620,C621,C630,C631,C632,C640,C641,C642,C643,C650,C651,C652,C653,C654,Qr0,Qr1,Qr2,Qr3,Qr4,Qr5,Qr6,S0,S1,S2]);
    same_ring=hom(KS,S_t,z->z,[t,z0,z1,z2,z3,z4,z5,z6])

    dimensions=[2,4,2,2,4,7,4,2,2,2,4,2,4,2,2]
    bol_hilb=true
    for (deg,cont_deg) in zip(deg_2,range(1,length(deg_2)))
        println("--------------------------- ",cont_deg," out of ",length(deg_2),". Check of degree ", deg)
        println("There are ",length(multiset_permutations(deg[2:8],7)) ," permutations")
        for deg_perm in multiset_permutations(deg[2:8],7)
            pushfirst!(deg_perm,deg[1])
            monomials=[]
            for (cou,i) in zip(couples,range(1,length(couples)))
                if deg_couples[i]==deg_perm
                    push!(monomials,same_ring(cou[2])*same_ring(cou[1]));
                end
            end
            if length(monomials)!=1
                monomials_mon=Set()
                for el in monomials
                    push!(monomials_mon,numerator(monomial(el,1)//t^(degree(el,t))))
                end
                if length(monomials_mon)!=dimensions[cont_deg]
                    println(deg_perm)
                    println(monomials_mon)
                    bol_hilb=false
                end
            end
        end
    end
    
    if !bol_hilb
        println("Not Sagbi")
        return false;
    end
    #Now we know it is Sagbi. We now compute the change of variables z_i -> y_i/x_i and the homogenization.     

    equations_mon=reduce(vcat, (exc_mon,hyps_mon,quad_mon,cub_mon,quart_mon,stra_mon));
    R, (t,x0,x1,x2,x3,x4,x5,x6,y0,y1,y2,y3,y4,y5,y6) = PolynomialRing(QQ, ["t","x0","x1","x2","x3","x4","x5","x6","y0","y1","y2","y3","y4","y5","y6"])
    x = R.([x0,x1,x2,x3,x4,x5,x6]);
    y = R.([y0,y1,y2,y3,y4,y5,y6]);

    exc_mon_R=[x[i] for i in  range(1,7)];
    hyps_mon_R,quad_mon_R,cub_mon_R,quart_mon_R,stra_mon_R=[],[],[],[],[];

    Toxy=hom(KS,FractionField(R),z->z,[t,y0//x0,y1//x1,y2//x2,y3//x3,y4//x4,y5//x5,y6//x6])

    cont=1;
    for i in 1:7
        for j in i+1:7
            for k in j+1:7
                push!(hyps_mon_R,numerator(Toxy(hyps_mon[cont])*x[1]*x[2]*x[3]*x[4]*x[5]*x[6]*x[7]//(x[i]*x[j]*x[k])));
                cont=cont+1;
            end
        end
    end

    cont=1;
    for i in 1:7
        for j in 1:7
            if j!=i
                push!(quad_mon_R,numerator(Toxy(quad_mon[cont])*x[1]*x[2]*x[3]*x[4]*x[5]*x[6]*x[7]*x[i]//x[j]));
                cont=cont+1;
            end
        end
    end

    cont=1;
    for i in 1:7
        for j in 1:i-1
            for k in 1:j-1
                push!(cub_mon_R,numerator(Toxy(cub_mon[cont])*x[1]*x[2]*x[3]*x[4]*x[5]*x[6]*x[7]*x[i]*x[j]*x[k]));
                cont=cont+1;
            end
        end
    end

    cont=1;
    for i in 1:7
        push!(quart_mon_R,numerator(Toxy(quart_mon[cont])*x[1]^2*x[2]^2*x[3]^2*x[4]^2*x[5]^2*x[6]^2*x[7]^2//x[i]));
        cont=cont+1;
    end

    cont=1;
    for i in 1:3
        push!(stra_mon_R,numerator(Toxy(stra_mon[cont])*x[1]*x[2]*x[3]*x[4]*x[5]*x[6]*x[7]));
        cont=cont+1;
    end

    #Finally we compute the matrix M encding the monomials of the initial forms of the generators.
    equations_mon_R=reduce(vcat, (exc_mon_R,hyps_mon_R,quad_mon_R,cub_mon_R,quart_mon_R,stra_mon_R));
    M=zero_matrix(ZZ,129,14);
    for (eq,row) in zip(equations_mon_R,range(1,129))
        for (var,col) in zip(R.([x0,x1,x2,x3,x4,x5,x6,y0,y1,y2,y3,y4,y5,y6]),range(1,14))
            M[row,col]=Oscar.degree(eq,var)
        end
    end
    
    return (true,M,equations_mon_R,deg_mon);
end

is_khovanskii (generic function with 1 method)

We run now the function on the matrix $A$. 

In [65]:
bol,M,mons,deg_mon=is_khovanskii(A)

Hyperplanes check 
Quad check 
Cub check 
Strange check 
--------------------------- 1 out of 15. Check of degree [1, 1, 0, 0, 1, 1, 1, 1]
There are 21 permutations
--------------------------- 2 out of 15. Check of degree [2, 1, 2, 1, 1, 1, 1, 1]
There are 7 permutations
--------------------------- 3 out of 15. Check of degree [2, 2, 2, 0, 1, 1, 1, 1]
There are 105 permutations
--------------------------- 4 out of 15. Check of degree [3, 3, 2, 2, 1, 1, 1, 1]
There are 105 permutations
--------------------------- 5 out of 15. Check of degree [3, 2, 2, 2, 2, 1, 1, 1]
There are 35 permutations
--------------------------- 6 out of 15. Check of degree [4, 2, 2, 2, 2, 2, 2, 2]
There are 1 permutations
--------------------------- 7 out of 15. Check of degree [4, 3, 1, 2, 2, 2, 2, 2]
There are 42 permutations
--------------------------- 8 out of 15. Check of degree [3, 2, 0, 1, 2, 2, 2, 2]
There are 42 permutations
--------------------------- 9 out of 15. Check of degree [4, 2, 1, 1, 3, 3, 2, 

(true, [1 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 1 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 1 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 1 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 1 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 1 0 0 0 0 0 0 0; 0 0 0 1 0 1 1 0 0 0 0 1 0 0; 0 0 0 0 1 1 1 0 0 1 0 0 0 0; 0 0 0 1 0 1 1 0 0 1 0 0 0 0; 0 0 1 1 0 0 1 0 0 0 0 1 0 0; 0 0 0 1 1 1 0 0 0 1 0 0 0 0; 0 1 0 0 1 1 0 0 0 0 0 0 0 1; 0 1 0 1 0 1 0 0 0 0 0 0 0 1; 0 1 0 1 0 0 1 0 0 0 0 1 0 0; 0 1 0 1 0 1 0 0 0 0 0 1 0 0; 0 1 1 0 0 1 0 0 0 0 0 0 0 1; 0 1 0 0 1 0 1 0 0 1 0 0 0 0; 0 1 0 0 1 1 0 0 0 1 0 0 0 0; 0 1 0 1 0 0 1 0 0 1 0 0 0 0; 0 1 0 1 0 1 0 0 0 1 0 0 0 0; 0 1 1 1 0 0 0 0 0 0 0 1 0 0; 0 0 0 0 1 1 1 1 0 0 0 0 0 0; 0 0 0 1 0 1 1 1 0 0 0 0 0 0; 1 0 0 1 0 0 1 0 0 0 0 1 0 0; 0 0 0 1 1 1 0 1 0 0 0 0 0 0; 1 0 0 0 0 1 1 0 0 1 0 0 0 0; 1 0 0 0 1 0 1 0 0 1 0 0 0 0; 0 0 1 0 1 1 0 1 0 0 0 0 0 0; 1 0 0 1 0 0 1 0 0 1 0 0 0 0; 0 0 1 1 0 1 0 1 0 0 0 0 0 0; 1 0 0 1 1 0 0 0 0 1 0 0 0 0; 1 1 0 0 0 1 0 0 0 0 0 0 0 1; 1 1 0 0 1 0 0 0 0 0 0 0 0 1; 0 1 0 

## Ehrhart-type formula

We now run the polyhedral computations described in Section 5 of the article.

We first introduce the unimodular transformation and apply it to the vectors $(a_1, \dots, a_7, b_1,b_3, b_5,b_7) \in \mathbb{Z}^{11}$ from the rows of $M$. 

In [66]:
Unimod_transf = matrix(ZZ,[0 0 0 0 0 0 0 1 1 1 1; 1 0 0 0 0 0 0 1 0 0 0; 0 1 0 0 0 0 0 0 0 0 0; 0 0 1 0 0 0 0 0 1 0 0; 0 0 0 1 0 0 0 0 0 0 0; 0 0 0 0 1 0 0 0 0 1 0; 0 0 0 0 0 1 0 0 0 0 0; 0 0 0 0 0 0 1 0 0 0 1; 0 0 0 0 0 0 0 0 1 0 0; 0 0 0 0 0 0 0 0 0 1 0; 0 0 0 0 0 0 0 0 0 0 1]);
M = M[:,[1,2,3,4,5,6,7,8,10,12,14]];

In [67]:
IM = zero_matrix(ZZ,129,11);
for i in 1:129 
    IM[i,:] = transpose(Unimod_transf*transpose(M[i,:]))
end

We then look at the graph $\Gamma$ defined by the new vectors. 

In [68]:
C = positive_hull(IM)

A polyhedral cone in ambient dimension 11

In [69]:
f_vector(C)

10-element Vector{fmpz}:
 126
 2016
 10839
 27198
 37635
 30891
 15286
 4416
 683
 48

In [70]:
facets(C)

48-element SubObjectIterator{LinearHalfspace{fmpq}} over the Halfspaces of R^11 described by:
4*x₁ - x₂ - 2*x₃ - x₄ - x₅ - x₆ - x₇ - 2*x₈ + x₁₁ ≦ 0
4*x₁ - x₂ - x₃ - x₄ - 2*x₅ - 2*x₆ - x₇ - x₈ + x₀₁ ≦ 0
2*x₁ - x₂ - x₄ - x₅ - x₆ - x₁₁ ≦ 0
-x₄ + x₉ ≦ 0
5*x₁ - 2*x₂ - x₃ - x₄ - x₅ - x₆ - 2*x₇ - x₈ - x₉ - x₀₁ - x₁₁ ≦ 0
2*x₁ - x₄ - x₅ - x₆ - x₇ - x₁₁ ≦ 0
2*x₁ - x₂ - x₄ - x₇ - x₈ - x₀₁ ≦ 0
6*x₁ - 2*x₂ - 2*x₃ - 2*x₅ - 2*x₆ - 2*x₇ - 2*x₈ - x₉ ≦ 0
x₁ - x₃ - x₄ - x₆ - x₈ + x₉ + x₀₁ + x₁₁ ≦ 0
4*x₁ - x₂ - 2*x₃ - 2*x₅ - 2*x₆ - x₇ - 2*x₈ + 2*x₀₁ + x₁₁ ≦ 0
2*x₁ - x₃ - x₅ - x₆ - x₇ - x₈ + x₀₁ ≦ 0
2*x₁ - 2*x₃ - x₄ - x₅ - x₆ - 2*x₈ + x₉ + 2*x₀₁ + 2*x₁₁ ≦ 0
⋮
5*x₁ - 2*x₂ - 2*x₃ - x₅ - x₆ - 2*x₇ - 2*x₈ - x₉ - x₀₁ + x₁₁ ≦ 0
2*x₁ - x₂ - x₇ - 2*x₉ - 2*x₀₁ - x₁₁ ≦ 0
2*x₁ - x₂ - x₃ - x₄ - x₇ - x₀₁ ≦ 0
2*x₁ - x₂ - x₃ - x₄ - x₇ - x₈ + x₁₁ ≦ 0
-x₃ - x₅ - x₈ + x₉ + 2*x₀₁ + 2*x₁₁ ≦ 0
-x₆ + x₀₁ ≦ 0
2*x₁ - x₂ - x₃ - x₇ - x₉ - 2*x₀₁ ≦ 0
4*x₁ - 2*x₂ - x₃ - x₄ - 2*x₇ - x₈ - x₉ - 2*x₀₁ ≦ 0
x₁ - x₂ - x₉ - x₀₁ - x₁₁ ≦ 0
2*x₁

### Mukai edge graph

We define the Mukai bilinear form in the case of the blow up of $\mathbb{P}^3$ in seven points. 

In [71]:
function Mukai_form(i,j)
    E=deg_mon[i]
    D=deg_mon[j]
    return (2*E[1]*D[1]-(E[1]-E[2])*(D[1]-D[2])-(E[1]-E[3])*(D[1]-D[3])-(E[1]-E[4])*(D[1]-D[4])-(E[1]-E[5])*(D[1]-D[5])-(E[1]-E[6])*(D[1]-D[6])-(E[1]-E[7])*(D[1]-D[7])-(E[1]-E[8])*(D[1]-D[8]));
end

Mukai_form (generic function with 1 method)

We check whether the polytope defined by $M$ has Mukai edge graph. The function takes as input the matrix $M$ and the degrees given as output `deg_mon` in the function `is_Khovanskii`. 

In [72]:
function has_Mukai_edge_graph(M,deg_mon)
    P=convex_hull(M);
    EdGr=Oscar.Graphs.edgegraph(P);
    E=Oscar.Graphs.edges(EdGr);

    graph_bol=true
    for vert in range(1,length(vertices(P)))
        cont_zero=0
        cont=0
        for vert2 in Graphs.neighbors(EdGr,vert)
            cont=cont+1
            if Mukai_form(vert,vert2)==0
                cont_zero=cont_zero+1
            end
        end
        println("For vertex ",vert," we get ",cont_zero,"/",cont, " have scalar product zero")
        if cont!=32
            graph_bol=false
        end
        if cont!=cont_zero
            graph_bol=false
        end
    end

    if graph_bol
        return true;
    end
    
    return false;
end

has_Mukai_edge_graph (generic function with 2 methods)

In [73]:
has_Mukai_edge_graph(M,deg_mon)

For vertex 1 we get 32/32 have scalar product zero
For vertex 2 we get 32/32 have scalar product zero
For vertex 3 we get 32/32 have scalar product zero
For vertex 4 we get 32/32 have scalar product zero
For vertex 5 we get 32/32 have scalar product zero
For vertex 6 we get 32/32 have scalar product zero
For vertex 7 we get 32/32 have scalar product zero
For vertex 8 we get 32/32 have scalar product zero
For vertex 9 we get 32/32 have scalar product zero
For vertex 10 we get 32/32 have scalar product zero
For vertex 11 we get 32/32 have scalar product zero
For vertex 12 we get 32/32 have scalar product zero
For vertex 13 we get 32/32 have scalar product zero
For vertex 14 we get 32/32 have scalar product zero
For vertex 15 we get 32/32 have scalar product zero
For vertex 16 we get 32/32 have scalar product zero
For vertex 17 we get 32/32 have scalar product zero
For vertex 18 we get 32/32 have scalar product zero
For vertex 19 we get 32/32 have scalar product zero
For vertex 20 we get 

true

### Input for Macaualy2 

To find the generators of the toric ideal in an efficient way, we can output the matrix $M$ so it is be given as input in Macaualy and then run the following commands. 

In [ ]:
function write_M_m2(M)
    print("matrix\"")
    for row in range(1,129)
        for col in range(1,14)
            print(M[row,col])
            if col!=14
                print(",")
            end
        end     
        print(";")
    end
    print("\"")
end

In [ ]:
M<---copy and paste output of write_M_m2(M)
M=transpose(M);
installPackage "FourTiTwo";
R=toricMarkov(M);
